In [1]:
import cv2
import mediapipe as mp
import numpy as np
import torch
from deepface import DeepFace
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image

emotion = [ "angry", "disgust", "fear", "happy", "neutral", "sad","surprise"]
seq_length = 30

from efficientnet_pytorch import EfficientNet
model_name = 'efficientnet-b0'  # b5

In [2]:
# 모델 불러오기

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # set gpu

model = EfficientNet.from_pretrained(model_name, num_classes=7)
model = model.to(device)

model.load_state_dict(torch.load('C:/Users/LIM HAYOUNG//Desktop/efficientNet_PER7_0501.pt',map_location ='cpu'))
model.eval()

Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [202]:
# image = np.array(cv2.imread('C:/Users/LIM HAYOUNG/Desktop/097-hap-03_copy.jpg'))
# image = cv2.resize(image, dsize=(224, 224),interpolation=cv2.INTER_LINEAR)
# image_swap = np.swapaxes(image, 0,2)
# image_swap = np.expand_dims(image_swap, axis=0)

# print(image_swap)

In [3]:
def make_prediction(image):
    
    emotion = [ "angry", "disgust", "fear", "happy", "neutral", "sad","surprise"]
    
    transfrom = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                ])

    tensor = torch.stack([transfrom(image)], dim=0)
    outputs =model(tensor)
    _, preds = torch.max(outputs, 1)
    prob = {}
    for i, em in enumerate(emotion)  :
        prob[em] = torch.softmax(outputs, dim=1)[0, i].item()
    return emotion[preds.item()],prob 


In [4]:
import pandas as pd

result_df = pd.DataFrame(columns= ['file_name','frame','angry','disgust','fear','happy','neutral','sad','surprise','True','Pred'])
result_df

,file_name,frame,angry,disgust,fear,happy,neutral,sad,surprise,True,Pred


In [24]:
result_df

,file_name,frame,angry,disgust,fear,happy,neutral,sad,surprise,True,Pred


In [ ]:
# 동영상 파일 반복하기

# 영상파일 경로
video_path = 'C:/Users/LIM HAYOUNG/Desktop/aihub_emotion_dataset/video'

# 얼굴을 찾고 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴 특징 그리기 위한 drawiong_utils 모듈 사용

sec = 1
for i in os.listdir(video_path) :
    print(i+' 시작')
    for j in os.listdir(video_path+'/'+i):
        # 얼굴 검출 신뢰도 기준 : 0.7
        cap = cv2.VideoCapture(video_path+'/'+i+'/'+j)
        label_vec = []
        with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
            confidendt_emotion='0'
            a =1
            while cap.isOpened():
                success, image = cap.read()

                if not success:
                    break

                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.
                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = face_detection.process(image)

                # Draw the face detection annotations on the image.
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                if results.detections:
                    for detection in results.detections:

                        #mp_drawing.draw_detection(image, detection) # keypoints dot

                        #gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

                        bbox = detection.location_data.relative_bounding_box
                        h, w, _ = image.shape 

                        real_bbox = {
                                'xmin':int(bbox.xmin * w),
                                'ymin':int(bbox.ymin * h),
                                'width':int(bbox.width * w),
                                'height':int(bbox.height * h)
                            }
#                         cv2.rectangle(image,(real_bbox['xmin'],real_bbox['ymin']),(real_bbox['xmin']+real_bbox['width'],real_bbox['ymin']+real_bbox['height']),
#                              (255,255,255),3)
                    roi_frame = image[real_bbox['ymin']:real_bbox['ymin']+real_bbox['height'],real_bbox['xmin']:real_bbox['xmin']+real_bbox['width']]
                    crop_image = Image.fromarray(roi_frame)
                    
            
            
#                     if a == 1 :
#                         pred, prob = make_prediction(crop_image)
#                         label = pred
#                         conf = prob[max(prob,key=prob.get)]
                    
                    # 5 frame 씩 감정 예측
                    if a % 5 == 0 :
                        pred, prob = make_prediction(crop_image)
                        label_vec.append(prob)
                    
                    # 30 frame(1초) 동안의 감정 예측 결과 신뢰도 평균 저장
                    if a % 30 == 0 :
                        angry = disgust = fear = happy = neutral = sad = surprise = 0

                        for m in label_vec :
                            angry += m['angry']
                            disgust += m['disgust']
                            fear += m['fear']
                            happy += m['happy']
                            neutral += m['neutral']
                            sad += m['sad']
                            surprise += m['surprise']


                        aver ={}

                        aver['angry'] = angry / len(label_vec)
                        aver['disgust'] = disgust / len(label_vec)
                        aver['fear'] = fear / len(label_vec)
                        aver['happy'] = happy / len(label_vec)
                        aver['neutral'] = neutral / len(label_vec)
                        aver['sad'] = sad / len(label_vec)
                        aver['surprise'] = surprise /len(label_vec)

                        label = max(aver,key = aver.get)
#                         conf = max(aver.values())
#                         print('file_name : {} / emotion : {} / conf : {} '.format(j, label,str(conf)))
#                         print(aver)
                        result_df.loc[sec] = [j,a,aver['angry'],aver['disgust'],aver['fear'],aver['happy'],aver['neutral'],aver['sad'],aver['surprise'],i,label]
#                         print(result_df)
                        sec+=1
                        label_vec = []

        #         cv2.putText(image,"{} : {}".format(pred,prob[max(prob,key=prob.get)]),(real_bbox['xmin'],real_bbox['ymin']),font,1,(255,0,0),2)
#                 cv2.putText(image,"{} : {}".format(label,conf),(real_bbox['xmin'],real_bbox['ymin']),font,1,(255,0,0),2)


#                 cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy =0.5))

                a+=1
                #if cv2.waitKey(5) & 0xFF == 27:
                if cv2.waitKey(1) == ord('q'): # 'q' 누르면 정지
                    break
    
    print(i + ' 완료 :'+ str(len(result_df)))
    cap.release()
    cv2.destroyAllWindows()

Anger 시작
Anger 완료 :372
Disgust 시작
Disgust 완료 :779
Fear 시작
Fear 완료 :1196
Happiness 시작
Happiness 완료 :1601
Neutral 시작


In [26]:
result_df

,file_name,frame,angry,disgust,fear,happy,neutral,sad,surprise,True,Pred
1,090-286.m2ts,30,0.107292,0.058354,0.179545,0.024308,0.0,0.043396,0.534569,Anger,surprise
2,090-286.m2ts,60,0.035383,0.032095,0.077364,0.003181,0.0,0.030438,0.811168,Anger,surprise
3,090-287.m2ts,30,0.138137,0.085682,0.167872,0.012595,0.0,0.049020,0.396021,Anger,surprise
4,090-287.m2ts,60,0.041227,0.020937,0.086595,0.004800,0.0,0.020691,0.812588,Anger,surprise
5,090-288.m2ts,30,0.039695,0.025813,0.157017,0.077174,0.0,0.038653,0.615350,Anger,surprise
...,...,...,...,...,...,...,...,...,...,...,...
368,099-284.m2ts,60,0.084443,0.313091,0.261697,0.002485,0.0,0.234683,0.102046,Anger,disgust
369,099-284.m2ts,90,0.025587,0.192026,0.354778,0.000745,0.0,0.316860,0.108854,Anger,fear
370,099-285.m2ts,30,0.122881,0.430638,0.183599,0.010865,0.0,0.206640,0.031408,Anger,disgust
371,099-285.m2ts,60,0.050858,0.295929,0.127029,0.019368,0.0,0.496545,0.006912,Anger,sad


In [9]:
# 얼굴을 찾고 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴 특징 그리기 위한 drawiong_utils 모듈 사용


# 동영상 파일 열기
# cap = cv2.VideoCapture(0) # 웹캠에서 작업
cap = cv2.VideoCapture('C:/Users/LIM HAYOUNG/Desktop/aihub_emotion_dataset/video/Anger/090-287.m2ts')

# model_selection : 0(카메라로부터 2m 이내 근거리 영상) or 1(5m 이내 거리 영상)
# min_detection_confidence : 얼굴 신뢰도 정도
font=cv2.FONT_HERSHEY_SIMPLEX


label_vec = []
with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.7) as face_detection:
    confidendt_emotion='0'
    a =1
    while cap.isOpened():
        success, image = cap.read()
        
        if not success:

            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:

                #mp_drawing.draw_detection(image, detection) # keypoints dot

                #gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

                bbox = detection.location_data.relative_bounding_box
                h, w, _ = image.shape 

                real_bbox = {
                        'xmin':int(bbox.xmin * w),
                        'ymin':int(bbox.ymin * h),
                        'width':int(bbox.width * w),
                        'height':int(bbox.height * h)
                    }
                cv2.rectangle(image,(real_bbox['xmin'],real_bbox['ymin']),(real_bbox['xmin']+real_bbox['width'],real_bbox['ymin']+real_bbox['height']),
                     (255,255,255),3)
            roi_frame = image[real_bbox['ymin']:real_bbox['ymin']+real_bbox['height'],real_bbox['xmin']:real_bbox['xmin']+real_bbox['width']]
            crop_image = Image.fromarray(roi_frame)
            if a == 1 :
                pred, prob = make_prediction(crop_image)
                label = pred
                conf = prob[max(prob,key=prob.get)]
                
            if a % 5 == 0 :
                pred, prob = make_prediction(crop_image)
                label_vec.append(prob)
#                 label = prob
#                 if prob[max(prob,key=prob.get)]>50 :
#                     confidendt_emotion = pred
            if a % 30 == 0 :
                angry = disgust = fear = happy = neutral = sad = surprise = 0
                
                for m in label_vec :
                    angry += m['angry']
                    disgust += m['disgust']
                    fear += m['fear']
                    happy += m['happy']
                    sad += m['sad']
                    surprise += m['surprise']
                    
                
                aver ={}
                
                aver['angry'] = angry / len(label_vec)
                aver['disgust'] = disgust / len(label_vec)
                aver['fear'] = fear / len(label_vec)
                aver['happy'] = happy / len(label_vec)
                aver['neutral'] = neutral / len(label_vec)
                aver['sad'] = sad / len(label_vec)
                aver['surprise'] = surprise /len(label_vec)
                
                label = max(aver,key = aver.get)
                conf = max(aver.values())
                print('emotion : {} / conf : {} '.format(label,str(conf)))
                label_vec = []
                
#         cv2.putText(image,"{} : {}".format(pred,prob[max(prob,key=prob.get)]),(real_bbox['xmin'],real_bbox['ymin']),font,1,(255,0,0),2)
        cv2.putText(image,"{} : {}".format(label,conf),(real_bbox['xmin'],real_bbox['ymin']),font,1,(255,0,0),2)
            
             
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy =0.5))
        
        a+=1
        #if cv2.waitKey(5) & 0xFF == 27:
        if cv2.waitKey(1) == ord('q'): # 'q' 누르면 정지
            break
            
cap.release()
cv2.destroyAllWindows()

emotion : surprise / conf : 0.4259236119687557 
emotion : surprise / conf : 0.8124517699082693 


In [5]:
label_vec

[]

In [ ]:
tensor = torch.from_numpy(image_swap).type(torch.FloatTensor)
outputs =model(tensor)
_, preds = torch.max(outputs, 1)

In [25]:
font=cv2.FONT_HERSHEY_SIMPLEX

In [45]:
prob

{'angry': 0.0008303597569465637,
 'disgust': 0.0009529745439067483,
 'fear': 0.0024136321153491735,
 'happy': 0.9868876338005066,
 'neutral': 0.0003204102977178991,
 'sad': 0.00847188662737608,
 'surprise': 0.00012304358824621886}

In [22]:
sorted(deepface_pred[0]['emotion'].items(),key = lambda item: item[1],reverse = True)

[('angry', 96.85723185539246),
 ('happy', 2.1006209775805473),
 ('neutral', 0.8104162290692329),
 ('sad', 0.2305970760062337),
 ('fear', 0.0010746959560492542),
 ('surprise', 6.0234776810830226e-05),
 ('disgust', 2.702072343652162e-06)]

In [87]:
# 얼굴을 찾고 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴 특징 그리기 위한 drawiong_utils 모듈 사용


# 동영상 파일 열기
# cap = cv2.VideoCapture(0) # 웹캠에서 작업
cap = cv2.VideoCapture('C:/Users/LIM HAYOUNG/Desktop/test_video.m2ts')

# model_selection : 0(카메라로부터 2m 이내 근거리 영상) or 1(5m 이내 거리 영상)
# min_detection_confidence : 얼굴 신뢰도 정도

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read() 
        if not success:

            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # deepface
        try : 
            deepface_pred = DeepFace.analyze(image, actions = ['emotion'])
            print(deepface_pred)
        except :
            continue
        
        
        rank_emotion = sorted(deepface_pred[0]['emotion'].items(),key = lambda item: item[1],reverse = True)    
        cv2.putText(image,str(rank_emotion[0]),(real_bbox['xmin'],real_bbox['ymin']),font,1,(255,0,0),2)
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy =0.5))
        
        #if cv2.waitKey(5) & 0xFF == 27:
        if cv2.waitKey(1) == ord('q'): # 'q' 누르면 정지
            breakq
            
cap.release()
cv2.destroyAllWindows()

Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.53it/s]


[{'emotion': {'angry': 12.266060709953308, 'disgust': 0.001594792956893798, 'fear': 1.1247393675148487, 'happy': 0.001494014486524975, 'sad': 43.88934373855591, 'surprise': 0.0002869443278541439, 'neutral': 42.71647930145264}, 'dominant_emotion': 'sad', 'region': {'x': 643, 'y': 213, 'w': 561, 'h': 561}}, {'emotion': {'angry': 1.6538159921765327, 'disgust': 5.62504975931688e-07, 'fear': 43.66461634635925, 'happy': 0.0001852071363828145, 'sad': 49.19450581073761, 'surprise': 0.0003847904736176133, 'neutral': 5.486489459872246}, 'dominant_emotion': 'sad', 'region': {'x': 743, 'y': 701, 'w': 296, 'h': 296}}]


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.83it/s]


[{'emotion': {'angry': 1.3774873688817024, 'disgust': 1.9529732497858276e-06, 'fear': 51.93573236465454, 'happy': 0.0003260754510847619, 'sad': 44.845691323280334, 'surprise': 0.0005333989065547939, 'neutral': 1.8402330577373505}, 'dominant_emotion': 'fear', 'region': {'x': 748, 'y': 703, 'w': 294, 'h': 294}}, {'emotion': {'angry': 13.917149434429474, 'disgust': 0.0008444624557474681, 'fear': 0.8466361012594579, 'happy': 0.0004190750087864221, 'sad': 50.7904886257066, 'surprise': 0.00011736957241385917, 'neutral': 34.444336957414514}, 'dominant_emotion': 'sad', 'region': {'x': 640, 'y': 211, 'w': 566, 'h': 566}}]


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.57it/s]


...

[{'emotion': {'angry': 1.0976606979966164, 'disgust': 5.875162578661275e-08, 'fear': 60.575151443481445, 'happy': 0.0008451863322989084, 'sad': 20.445556938648224, 'surprise': 0.0008340350177604705, 'neutral': 17.879952490329742}, 'dominant_emotion': 'fear', 'region': {'x': 739, 'y': 695, 'w': 327, 'h': 327}}, {'emotion': {'angry': 0.13291750801727176, 'disgust': 0.0836852821521461, 'fear': 56.59080147743225, 'happy': 0.04440386546775699, 'sad': 30.278366804122925, 'surprise': 0.0005622721346298931, 'neutral': 12.869267165660858}, 'dominant_emotion': 'fear', 'region': {'x': 677, 'y': 207, 'w': 555, 'h': 555}}]


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.45it/s]


[{'emotion': {'angry': 2.5350768119096756, 'disgust': 4.2633527819191386e-07, 'fear': 40.77889621257782, 'happy': 0.00036304500099504367, 'sad': 46.43335044384003, 'surprise': 0.0006309218406386208, 'neutral': 10.251682251691818}, 'dominant_emotion': 'sad', 'region': {'x': 745, 'y': 702, 'w': 315, 'h': 315}}, {'emotion': {'angry': 0.2418952528387308, 'disgust': 0.0010451478374307044, 'fear': 7.798930257558823, 'happy': 0.0705393438693136, 'sad': 52.28107571601868, 'surprise': 0.32946730498224497, 'neutral': 39.27704989910126}, 'dominant_emotion': 'sad', 'region': {'x': 673, 'y': 205, 'w': 561, 'h': 561}}]


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.14it/s]


[{'emotion': {'angry': 3.302347266394226, 'disgust': 5.624730232744112e-07, 'fear': 47.46791154528719, 'happy': 0.0008932200061498109, 'sad': 37.78493550003344, 'surprise': 0.0013890502650014274, 'neutral': 11.442519850430507}, 'dominant_emotion': 'fear', 'region': {'x': 743, 'y': 702, 'w': 315, 'h': 315}}, {'emotion': {'angry': 0.49365727527824343, 'disgust': 0.00042784832301452787, 'fear': 7.173270097168696, 'happy': 0.014192868784710023, 'sad': 27.16092493068277, 'surprise': 0.1905975508470894, 'neutral': 64.96693074309}, 'dominant_emotion': 'neutral', 'region': {'x': 675, 'y': 205, 'w': 562, 'h': 562}}]


In [86]:
str(rank_emotion[0])

"('sad', 43.88934373855591)"